In [0]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)
imdb = keras.datasets.imdb

#(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [0]:
import json
import collections
def ReadListAndDictFromFile(readFileName):
    with open(readFileName, "r",encoding='UTF-8') as f:
        readedList =  json.loads(f.read())
        return readedList
        
#for example
CNN_list = ReadListAndDictFromFile("CNN_sentence_list.txt")
label_list = ReadListAndDictFromFile("label_list.txt")
print(CNN_list[0][1])
minLength = 20
with open('sorted_word2id.txt', 'r') as fp:
    word2id = json.loads(fp.read())
word_dict = dict(collections.Counter(word2id).most_common()[:-10000-1:-1])

new_CNN_list = []
new_label_list = []
for i in range(len(label_list)):
    for j in range(len(CNN_list[i])):
        if len(CNN_list[i][j]) < minLength:
            continue
        else:    
            for k in range(len(CNN_list[i][j])):
                if CNN_list[i][j][k] in word_dict:
                    CNN_list[i][j][k] = word_dict[CNN_list[i][j][k]]+3
                else:
                    CNN_list[i][j][k] = 2
            new_CNN_list.append(CNN_list[i][j])
            if label_list[i] < 2.1:
                new_label_list.append(0.0)
            elif label_list[i] < 3.1:
                new_label_list.append(1.0)  
            else:
                new_label_list.append(2.0)

In [0]:
def reCount():
    new_CNN_list = []
    new_label_list = []
    for i in range(len(label_list)):
        for j in range(len(CNN_list[i])):
            if len(CNN_list[i][j]) < minLength:
                continue
            else:    
                for k in range(len(CNN_list[i][j])):
                    if CNN_list[i][j][k] in word_dict:
                        word_dict[CNN_list[i][j][k]] += 1
                    else:
                        word_dict[CNN_list[i][j][k]] =1
                new_CNN_list.append(CNN_list[i][j])
                if label_list[i] < 2.1:
                    new_label_list.append(0.0)
                elif label_list[i] < 4.1:
                    new_label_list.append(1.0)  
                else:
                    new_label_list.append(2.0)  
def ordered_list(word_dict):    
    cnt = collections.Counter(word_dict)
    codex = {}
    print(dict(cnt.most_common(100)))
    words_ord = list(dict(cnt))
    for k, i in enumerate(dict(cnt.most_common()).keys()):
        codex[i] = k
    with open('sorted_word2id.txt', 'w') as fp:
        json.dump(codex, fp)

In [0]:
train_data = new_CNN_list[2000:100000]
train_labels = new_label_list[2000:100000]
test_data = new_CNN_list[:2000]
test_labels = new_label_list[:2000]

In [0]:
# A dictionary mapping words to an integer index
#word_index = {str(val):int(key) for (key, val) in word_dict.items()}
word_index = word_dict
# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
#word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
decode_review(test_data[0])

In [0]:
import matplotlib.pyplot as plt

# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=[len(s)/10 for s in train_data], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('My Very Own Histogram')
plt.text(23, 45, r'$\mu=15, b=3$')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 100) * 100 if maxfreq % 100 else maxfreq + 100)


In [0]:
vocab_size = 10000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_LENGTH = 100
RNN_UNITS = 100
BATCH_SIZE = 128
CLASSES = 3
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=MAX_SEQUENCE_LENGTH)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=MAX_SEQUENCE_LENGTH)

x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = keras.utils.to_categorical(train_labels[:10000])
partial_y_train = keras.utils.to_categorical(train_labels[10000:])
# input shape is the vocabulary count used for the movie reviews (10,000 words)
print(partial_x_train.shape, y_val.shape, train_data.shape)


In [0]:
lstm_model = keras.Sequential()
lstm_model.add(keras.layers.Embedding(vocab_size, EMBEDDING_LENGTH))
lstm_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units = RNN_UNITS)))
lstm_model.add(tf.keras.layers.Dropout(0.5))
lstm_model.add(keras.layers.Dense(16, activation=tf.nn.relu))
lstm_model.add(keras.layers.Dense(CLASSES, activation=tf.nn.sigmoid))
lstm_model.compile(optimizer=tf.train.AdamOptimizer(),              loss='binary_crossentropy',              metrics=['accuracy'])

In [0]:
sequence_input = keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

embedded_sequences = keras.layers.Embedding(vocab_size, EMBEDDING_LENGTH)(sequence_input)

tf.expand_dims(embedded_sequences, 3)

#cnn = tf.keras.layers.Conv2D(100, (3, EMBEDDING_LENGTH), (1, 1), activation = 'relu', kernel_regularizer = keras.regularizers.l2(3))(embedded_sequences)
#dropout = tf.keras.layers.Dropout(0.5)(cnn)
preds = keras.layers.Dense(CLASSES, activation='softmax')(embedded_sequences)
cnn_model = tf.keras.Model(sequence_input, preds)
#cnn_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
#cnn_model.summary()

In [0]:
#lstm_model.add(tf.keras.layers.Conv2D(100, (3, 64), (1, 1), activation = 'relu', kernel_regularizer = keras.regularizers.l2(3)))
sequence_input = keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = keras.layers.Embedding(vocab_size, EMBEDDING_LENGTH)(sequence_input)
l_cov1= keras.layers.Conv1D(EMBEDDING_LENGTH, 3, activation='relu')(embedded_sequences)
l_pool1 = keras.layers.MaxPooling1D(5)(l_cov1)
l_cov2 = keras.layers.Conv1D(EMBEDDING_LENGTH, 3, activation='relu')(l_pool1)
l_pool2 = keras.layers.MaxPooling1D(5)(l_cov2)
#l_cov3 = keras.layers.Conv1D(EMBEDDING_LENGTH, 3, activation='relu')(l_pool2)
#l_pool3 = keras.layers.MaxPooling1D(3)(l_cov3)  # global max pooling
l_flat = keras.layers.Flatten()(l_pool2)
l_dense = keras.layers.Dense(EMBEDDING_LENGTH, activation='relu')(l_flat)
preds = keras.layers.Dense(CLASSES, activation='softmax')(l_dense)
akshat_cnn_model = tf.keras.Model(sequence_input, preds)
#akshat_cnn_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
#akshat_cnn_model.summary()

In [0]:
sequence_input = keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = keras.layers.Embedding(vocab_size, EMBEDDING_LENGTH)(sequence_input)
l_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(RNN_UNITS))(embedded_sequences)
preds = keras.layers.Dense(CLASSES, activation='softmax')(l_lstm)
richard_model = tf.keras.Model(sequence_input, preds)
#richard_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
#richard_model.summary()

In [0]:
history = lstm_model.fit(partial_x_train,
                    partial_y_train,
                    epochs=3,
                    batch_size=BATCH_SIZE,
                    validation_data=(x_val, y_val),
                    verbose=1
                    )

## Evaluate the model

And let's see how the model performs. Two values will be returned. Loss (a number which represents our error, lower values are better), and accuracy.

In [0]:
results = lstm_model.evaluate(test_data, keras.utils.to_categorical(test_labels))

print(results)

In [0]:
import numpy as np
np.set_printoptions(threshold=np.nan)
f = open("output-predict",'w')  
f2 = open("output-target",'w')
predict = lstm_model.predict(tf.convert_to_tensor(x_val), steps = 1)
target = y_val
print (predict, file=f)  
print (target, file=f2)
f.close()
f2.close()

predict = np.transpose(predict)
target = np.transpose(target)
tp = [np.sum(np.dot(p, t)) for p, t in zip(predict, target)]
tn = [np.sum(np.dot((1-p), (1-t))) for p, t in zip(predict, target)]
fp = [np.sum(np.dot(p, (1-t))) for p, t in zip(predict, target)]
fn = [np.sum(np.dot((1-p), t)) for p, t in zip(predict, target)]
eva = np.array([tp,tn,fp,fn])
precision = np.divide(tp, np.add(tp, fp))
recall  = np.divide(tp, np.add(tp, fn))
f1_score = (2/((1/precision)+(1/recall)))
result = np.array([precision,recall,f1_score])
print(eva)
print(result)


This fairly naive approach achieves an accuracy of about 87%. With more advanced approaches, the model should get closer to 95%.

## Create a graph of accuracy and loss over time

`model.fit()` returns a `History` object that contains a dictionary with everything that happened during training:

In [0]:
history_dict = history.history
history_dict.keys()

There are four entries: one for each monitored metric during training and validation. We can use these to plot the training and validation loss for comparison, as well as the training and validation accuracy:

In [0]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [0]:
plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


In this plot, the dots represent the training loss and accuracy, and the solid lines are the validation loss and accuracy.

Notice the training loss *decreases* with each epoch and the training accuracy *increases* with each epoch. This is expected when using a gradient descent optimization—it should minimize the desired quantity on every iteration.

This isn't the case for the validation loss and accuracy—they seem to peak after about twenty epochs. This is an example of overfitting: the model performs better on the training data than it does on data it has never seen before. After this point, the model over-optimizes and learns representations *specific* to the training data that do not *generalize* to test data.

For this particular case, we could prevent overfitting by simply stopping the training after twenty or so epochs. Later, you'll see how to do this automatically with a callback.